In [7]:
from mbi import mechanism, FactoredInference
import numpy as np
import argparse
from hdmm import workload, templates, error
from mbi import Dataset
import itertools
from autodp import privacy_calibrator
import os

def convert_matrix(domain, cliques):
    weights = {}
    for proj in cliques:
        tpl = tuple([domain.attrs.index(i) for i in proj])
        weights[tpl] = 1.0
    return workload.Marginals.fromtuples(domain.shape, weights)

def convert_back(domain, Q):
    result = []
    for Qi in Q.matrices:
        wgt = Qi.weight
        key = tuple([domain.attrs[i] for i in Qi.base.tuple()])
        result.append((key, wgt))
    return result


In [ ]:
prng = np.random
number = 1
prefix = os.getenv('HD_DATA') #'/home/rmckenna/Repos/hd-datasets/clean/'
attrs = ['ALS Unit', 'Battalion', 'Call Final Disposition', 'Call Type', 'Call Type Group', 'City', 'Final Priority', 'Fire Prevention District', 'Neighborhooods - Analysis Boundaries', 'Original Priority', 'Priority', 'Station Area', 'Supervisor District', 'Unit Type', 'Zipcode of Incident']

data = Dataset.load(prefix+'fire.csv', prefix+'fire-domain.json').project(attrs)

all3way = list(itertools.combinations(data.domain, 3))
cliques = [all3way[i] for i in prng.choice(len(all3way), number, replace=False)]
W = convert_matrix(data.domain, cliques)

best_obj = np.inf
restarts = 1
for _ in range(restarts):
    temp = templates.Marginals(data.domain.shape, approx=False)
    obj = temp.optimize(W)
    if obj < best_obj:
        best_obj = obj
        A = temp.strategy()
